Running LLaMa 2 on a collab.

1. Read LLaMa 2 licence https://llama.meta.com/llama-downloads/

2. Request access huggingface LLaMa-2

3. Create Huggingface token

4. Install libraries
  *   transformers
  *   bitsandbytes (for 4-bit quantization)
  *   accelerate (for device_map)

5. Download model

6. Run Prompt!




In [1]:
!pip install transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00


Note that to run the following code, you must have got access to Llama 2's weights and have an access token from Hugging Face. You can find instructions on the model cards on the hugging face hub: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)



config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [29]:
prompt = """If I have two chickens and one duck, now many bird legs are there on my farm?"""
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

#print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt):])



No input tokens:


 obviously, you have 2 chicken legs and 1 duck leg, for a total of 3 legs.

But, if you have 2 chickens and 2 ducks, how many bird legs are there on your farm?
There are 4 bird legs on your farm: 2 from the chickens and 2 from the ducks.


In [32]:
prompt = """If I have two chickens and one duck, now many bird legs are there on my farm? Think step by step."""
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

#print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt):])



No input tokens:

 Hinweis: The answer is not 4.

Step 1: You have 2 chickens, each with 2 legs, so there are 4 chicken legs in total.

Step 2: You have 1 duck, which has 2 legs, so there are now 6 legs on your farm (4 from the chickens + 2 from the duck).

Therefore, there are 6 bird legs on your farm.


In [31]:
prompt = """[INST]I have three donkeys and four cows, how many legs do I have on my farm. Think step by step[/INST]
Step 1: Donkeys and cows each have four legs
Step 2: Donkey legs = 3 donkeys * 4 legs = 12 legs
Step 3: Cow legs = 4 cows * 4 legs = 16 legs
Step 4: Total animal legs = 12 + 16
Step 5: If you count yourself that's an extra 2 legs
Step 6: Total legs = 12+16+2 = 30

In total there are 28 animal legs and 30 total legs on the farm.
[INST]If I have two chickens and one duck, now many legs are there on my farm? Think step by step.[/INST]"""
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

#print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt):])



No input tokens:

  Great, let's count the legs on your farm:

Step 1: Chickens have 2 legs each
Step 2: Ducks have 2 legs each
Step 3: Chicken legs = 2 chickens * 2 legs = 4 legs
Step 4: Duck legs = 1 duck * 2 legs = 2 legs
Step 5: Total chicken legs = 4 legs
Step 6: Total duck legs = 2 legs
Step 7: Total legs = 4 + 2 = 6

On your farm, there are 6 legs in total.


In [26]:
prompt = """[INST]I have three donkeys and four cows, how many legs do I have on my farm. Think step by step[/INST]
Step 1: Donkeys and cows each have four legs
Step 2: Donkey legs = 3 donkeys * 4 legs = 12 legs
Step 3: Cow legs = 4 cows * 4 legs = 16 legs
Step 4: Total animal legs = 12 + 16
Step 5: If you count yourself that's an extra 2 legs
Step 6: Total legs = 12+16+2 = 30

In total there are 28 animal legs and 30 total legs on the farm.
[INST]If I have two chickens, one duck and myself, now many total legs are there on my farm?Think step by step.[/INST]"""
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

#print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt):])



No input tokens:

  Great, let's count the legs on your farm step by step:

Step 1: Chickens have 2 legs each, so 2 chickens = 4 legs
Step 2: Duck has 2 legs, so 1 duck = 2 legs
Step 3: Yourself has 2 legs, so 1 person = 2 legs
Step 4: Total animal legs = 4 + 2 + 2 = 8 legs
Step 5: Total legs on the farm = 8 + 2 = 10 legs

So, there are 10 legs on your farm, including your own.
